# Nonlinear Eulerian elasticity using the inverse map

**Description:** This is a collection of nonlinear PDEs for Lagrangian/Eulerian hyperelastic solids and ultimately for a two-phase model describing the travelling porosity wave phenomenon. 

**Authors:** Andrea Zafferi, Dirk Peschka

## 1. Stationary Setting

### Lagrangian formulation

Nonlinear elasicity is based on a flow map $\bar{\chi}:\bar{\Omega}\to\mathbb{R}^d$, where $\bar{\Omega}\subset\mathbb{R}^d$ is a reference domain and 

$$
\Omega = \bar{\chi}(\bar{\Omega})=\{x=\bar{\chi}(\bar{x}):\bar{x}\in\bar{\Omega}\},
$$

denotes the deformed domain. For simplicity we will assume $\Omega=\bar{\Omega}$ throughout this work. Then, for a nonlinear hyperelastic problem with deformation gradient

$$
\bar{\boldsymbol{F}}=\bar{\nabla}\bar{\chi},
$$

the observed deformation emerges from the minimization of an energy

$$
\bar{H}(\bar{q}) = \int_{\bar{\Omega}} W_\mathrm{elast}(\bar{\boldsymbol{F}})\,\mathrm{d}\bar{x},
$$

over $\bar{q}=\bar{\chi}$ from a suitable function space. For practical reasons we rather work with the displacement $\bar{u}(\bar{x})=\bar{\chi}(\bar{x})-\bar{x}$.

###  Eulerian formulation

Assuming $\bar{\chi}$ in invertible, then for any given function $\bar{f}$ defined on $\bar{\Omega}$ we can introduce a corresponding function $f$ defined on $\Omega$ by

$$
f(x)=\bar{f}(\bar{\chi}^{-1}(x)).
$$

If we want to rewrite the energy minimization above in terms of Eulerian functions, then the flow map itself is not useful since $\chi=\bar{\chi}(\bar{\chi}^{-1}(x))=x$ is not a viable unknown function defined on $\Omega$. Instead we can use the return map (inverse map)

$$
\alpha(x) = \bar{\chi}^{-1}(x),
$$

which allows us to express

$$
\boldsymbol{F} = (\nabla\alpha)^{-1}(x),
$$

and using $q=\alpha$ formulate a corresponding energy minimization of 

$$
H(q) = \int_{\Omega} \frac{1}{J}W_\mathrm{elast}(\boldsymbol{F})\,\mathrm{d}x,
$$

where $J=\det(\boldsymbol{F})$. Also here, for practical resons we rather work with the displacement $u(x)=\bar{u}(\chi^{-1}(x))=x-\alpha(x)$.


### CODE BLOCK 1: Solver for the stationary setting

Description: This is the code for the stationary problem, where all the parameters, but in particular the number of mesh points in each direction `k` or the polynomial degree `FE_deg` can be adjusted.

In [ ]:
from fenics import *
from mshr import *
import numpy as np
from pylab import plt
import logging
import csv

from utils import *

logging.getLogger('FFC').setLevel(logging.WARNING)
parameters["form_compiler"]["quadrature_degree"] = 5

# ------------------------------------
# Setup model parameters
# ------------------------------------

k           = 16    # mesh points
mu          = 1.0   # Lame constant
lmbda       = 2.0   # Lame constant
f0          = 6.0   # gravity

dim         = 2     # spatial dimension
FE_deg      = 1
FE          = VectorElement('P',triangle,FE_deg)

# ------------------------------------
# Setup Lagrangian/Eulerian mesh, function space, Dirichlet bcs
# ------------------------------------

meshbar     = UnitSquareMesh(k,k) # Generation of Lagrangian mesh
mesh        = UnitSquareMesh(k,k) # Generation of Eulerian mesh
    
Qbar  = get_space(meshbar,FE)   # Lagrangian state space 
Q     = get_space(mesh   ,FE)   # Eulerian state space


bcbar = DirichletBC(Qbar, Constant((0, 0)) , 'on_boundary')
bc    = DirichletBC(Q   , Constant((0, 0)) , 'on_boundary')
f     = Expression(('0','f0'),f0=f0,degree=2)

# ------------------------------------
# Lagrangian energy
# ------------------------------------
def energybar(qbar):
    xbar = SpatialCoordinate(meshbar)
    I    = Identity(dim)
    Fbar = I + grad(qbar)
    Cbar = Fbar.T*Fbar
    Jbar = det(Fbar)
    Ebar = mu/2*(tr(Cbar-I) - 2*ln(Jbar)) + lmbda/2*(Jbar-1)**2 + inner(f,xbar+qbar)
    return Ebar

# ------------------------------------
# Eulerian energy
# ------------------------------------
def energy(q):
    x = SpatialCoordinate(mesh)
    I = Identity(dim)
    F = inv(I - grad(q))
    C = F.T*F
    J = det(F)
    E = ( mu/2*(tr(C-I) - 2*ln(J)) + lmbda/2*(J-1)**2 + inner(f,x) ) / J
    return E

# ------------------------------------
# Solve Lagrangian problem
# ------------------------------------
qbar   = Function(Qbar)
Ebar   = energybar(qbar)*dx
Resbar = derivative(Ebar,qbar)

solve(Resbar==0,qbar,bcbar,solver_parameters={'newton_solver': {'linear_solver': 'mumps'}})

# ------------------------------------
# Solve Eulerian problem
# ------------------------------------
q   = Function(Q)
E   = energy(q)*dx
Res = derivative(E,q)

solve(Res==0,q,bc,solver_parameters={'newton_solver': {'linear_solver': 'mumps'}})

# ------------------------------------
# Plotting and postprocessing
# ------------------------------------
def plot_sol(q_,mesh_):
    plt.figure(figsize=(scal*6,scal*4))
    plt.rcParams.update({'font.size': 14})
    cbar = plot(inner(qbar,qbar))

    if FE_deg==1:
        V      = FunctionSpace(mesh_,'CG',1)
        xbar   = V.tabulate_dof_coordinates()
        q_x = q_.sub(0, deepcopy=True)
        q_y = q_.sub(1, deepcopy=True)
        plt.quiver(xbar[:,0],xbar[:,1],q_x.vector()[:],q_y.vector()[:],color='w',scale=1)
    else:
        plot(q)
    plot(meshbar,linewidth=0.6)
    plt.colorbar(cbar, fraction=0.046, pad=0.04)

ALE.move(meshbar,qbar)
scal = 1

plot_sol(qbar,meshbar)
plt.savefig('media/stationary_sol_Lagrange.png',dpi=300,bbox_inches = 'tight')
plot_sol(q,mesh)
plt.savefig('media/stationary_sol_Euler.png',dpi=300,bbox_inches = 'tight')

# ------------------------------------
# Compute L2,H1,Linf errors (difference Lagrangian vs Eulerian)
# ------------------------------------
qbari = project(qbar     ,Q,bcs=bc,solver_type='cg')
erri  = project(q - qbari,Q,bcs=bc,solver_type='cg')

L2err  = norm(erri, 'l2')
H1err  = norm(erri, 'h1')
Lierr  = norm(erri.vector(), 'linf')

print('Difference between Lagrangian and Eulerian solution:\n')
print(f'# k={k:3} L2: {L2err:.4e} H1: {H1err:.4e} Li: {Lierr:.4e}')

### CODE BLOCK 2: Plotter for errors

Description: This plots the errors (differences between Lagrangian and Eulerian solutions) obtained by CODE BLOCK 1.

In [ ]:
import numpy as np
from pylab import plt

# convergence for P1 displacements
# k=  2 L2: 9.9856e-03 H1: 5.7363e-02 Li: 2.6402e-02
# k=  4 L2: 6.4581e-03 H1: 4.4343e-02 Li: 1.2488e-02
# k=  8 L2: 2.3239e-03 H1: 2.5729e-02 Li: 6.3256e-03
# k= 16 L2: 9.1196e-04 H1: 1.2973e-02 Li: 3.0733e-03
# k= 32 L2: 3.0661e-04 H1: 6.6898e-03 Li: 1.5251e-03
# k= 64 L2: 8.1549e-05 H1: 2.9744e-03 Li: 5.2709e-04
# k=128 L2: 2.1233e-05 H1: 1.4093e-03 Li: 1.7999e-04
# k=256 L2: 5.4107e-06 H1: 6.8845e-04 Li: 6.5984e-05
# k=512 L2: 1.3706e-06 H1: 3.3977e-04 Li: 2.1064e-05

# convergence for P2 displacements
# k=  2 L2: 1.2401e-02 H1: 1.1854e-01 Li: 3.5739e-02
# k=  4 L2: 4.4063e-03 H1: 6.8382e-02 Li: 2.0545e-02
# k=  8 L2: 1.2282e-03 H1: 2.7894e-02 Li: 7.8188e-03
# k= 16 L2: 3.1654e-04 H1: 1.1182e-02 Li: 2.4954e-03
# k= 32 L2: 7.4658e-05 H1: 5.2901e-03 Li: 7.5060e-04
# k= 64 L2: 1.9541e-05 H1: 2.4590e-03 Li: 2.2155e-04
# k=128 L2: 4.8303e-06 H1: 1.1985e-03 Li: 6.4999e-05
# k=256 L2: 1.2119e-06 H1: 5.8740e-04 Li: 1.9033e-05
# k=512 L2: 3.0127e-07 H1: 2.9089e-04 Li: 5.5969e-06

k = [2,4,8,16,32,64,128,256,512]
h = np.divide(1,k)
# P1
L2_errs = [9.9856e-03,6.4581e-03,2.3239e-03,9.1196e-04,3.0661e-04,8.1549e-05,2.1233e-05,5.4107e-06,1.3706e-06]
H1_errs = [5.7363e-02,4.4343e-02,2.5729e-02,1.2973e-02,6.6898e-03,2.9744e-03,1.4093e-03,6.8845e-04,3.3977e-04]
Li_errs = [2.6402e-02,1.2488e-02,6.3256e-03,3.0733e-03,1.5251e-03,5.2709e-04,1.7999e-04,6.5984e-05,2.1064e-05]

# P2
L2_errs = [1.2401e-02,4.4063e-03,1.2282e-03,3.1654e-04,7.4658e-05,1.9541e-05,4.8303e-06,1.2119e-06,3.0127e-07]
H1_errs = [1.1854e-01,6.8382e-02,2.7894e-02,1.1182e-02,5.2901e-03,2.4590e-03,1.1985e-03,5.8740e-04,2.9089e-04]
Li_errs = [3.5739e-02,2.0545e-02,7.8188e-03,2.4954e-03,7.5060e-04,2.2155e-04,6.4999e-05,1.9033e-05,5.5969e-06]

plt.figure(figsize=(6,5))

plt.rcParams.update({'font.size': 14})

plt.loglog(h,L2_errs,'+',label='$L^2$',markersize=8)
plt.loglog(h,H1_errs,'x',label='$H^1$',markersize=8)
plt.loglog(h,Li_errs,'>',label='$L^\infty$',markersize=8)

L2scal = L2_errs[-1]/h[-1]**2
H1scal = H1_errs[-1]/h[-1]
Liscal = Li_errs[-1]/h[-1]**1.75

plt.plot(h,np.multiply(np.power(h,1),H1scal),':',color=[0.5,0.5,0.5],label='$\sim h$',linewidth=2)
plt.plot(h,np.multiply(np.power(h,2),L2scal),'--',color=[0.5,0.5,0.5],label='$\sim h^2$',linewidth=2)
# plt.plot(h,np.multiply(np.power(h,1.75),Liscal),'--',color=[0.5,0.5,0.5])
plt.xlim([1e-3,1])
plt.legend()
plt.xlabel('mesh size $h$')
plt.ylabel('error ($P_2$ displacements)')

plt.grid(True)

plt.savefig('media/stationary_convergence_P2.pdf',bbox_inches = 'tight')